In [1]:
import subprocess
import pandas as pd
import numpy as np
import requests
import json
import os
import paho.mqtt.client as client
from pathlib import Path
from tqdm import tqdm
import traceback
from sklearn.model_selection import train_test_split

In [2]:
# def process_bin_clas(df_train, df_test):
#     df_total = pd.concat([df_train, df_test], axis=0)
#     tar = df_total.iloc[:, -1]
#     df_total = df_total.iloc[:, :-1]
#     cat = df_total.select_dtypes(include=['object']).columns.to_list()
#     df_total = pd.get_dummies(df_total, cat)
#     df_total = pd.concat([df_total, tar], axis=1)
#     df_train = df_total.iloc[:df_train.shape[0],:]
#     df_test = df_total.iloc[df_train.shape[0]:,:]
    
#     return df_train, df_test

# def get_missing():
#     missingValueColumns = datafrm.columns[datafrm.isnull().any()].tolist()
#     percent_missing = datafrm[missingValueColumns].isnull().sum()
#     print("Missing value count columnwise:")
#     print(percent_missing)


# data_dir =\
#     r"C:\Users\arpit\workspace\setup-stuff\gateway_and_dataset\Classification\DATASETS_CLASSIFICATION\BINARY_PROBLEMS"
# datasets = ["Adult", "Breast Cancer", "Credit Screening", "Ionosphere", "Liver Disorder", "Pima Indian", "Sonar"]
# shorts = ["AD", "BC", "CR", "IO", "LD", "PI", "SN"]
# seps = [",", "\t", "\t", "\t", "\t", "\t", ","]
# header = [False, False, False, False, False, False, False]

# seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# for i in range(len(datasets)):
#     for seed in tqdm(seeds):
#         dataset = datasets[i]
#         short = shorts[i]
        
#         csv_path_train = os.path.join(data_dir, dataset, short + 'Train' + str(seed) + 'N.txt')
#         csv_path_test = os.path.join(data_dir, dataset, short + 'Test' + str(seed) + 'N.txt')
#         if header[i]:
#             df_train = pd.read_csv(csv_path_train, sep = seps[i])
#             df_test = pd.read_csv(csv_path_test, sep = seps[i])
#         else:
#             df_train = pd.read_csv(csv_path_train, sep = seps[i], header=None)
#             df_test = pd.read_csv(csv_path_test, sep = seps[i], header=None)

#         df_train, df_test = process_bin_clas(df_train, df_test)
#         x, y = df_train.iloc[: , :-1], df_train.iloc[: , -1]
#         x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=4, stratify=y)
#         train_part_k = pd.concat([x_train, y_train], axis=1)
#         train_part_rest = pd.concat([x_test, y_test], axis=1)
        
#         train_part_k.to_csv(os.path.join(data_dir, dataset, short + 'Train' + str(seed) + 'N_mod.txt'), index=False)
#         train_part_rest.to_csv(os.path.join(data_dir, dataset, short + 'Val' + str(seed) + 'N_mod.txt'), index=False)
#         df_test.to_csv(os.path.join(data_dir, dataset, short + 'Test' + str(seed) + 'N_mod.txt'), index=False)
    

In [2]:
def get_prereq_results(target):
    body = {"algorithm": "classification", "target": target}
    url = 'http://localhost:5000/pre_req'
    r = requests.post(url, json=body)
    prereq_res = r.json()
#     print(prereq_res.keys())
    return prereq_res

In [3]:
def get_phase0_results():
    body = {"algorithm": "classification","chunk_count": 30}
    url = 'http://localhost:5000/phase0'
    r = requests.post(url, json=body)
    phase0_res = r.json()
#     print(phase0_res.keys())
    return phase0_res

In [4]:
def get_phase1_results(n_c=2, training_csv=None):
    body = {
        "algorithm": "classification",
        "Num_Classes": n_c,
        "NUM_ITERATIONS": 4,
        "NET_LATTICE_PHASE_1": [3, 4, 5, 6, 7, 8],
        "no_data_Passes_kernel_Phase_1": 10,
        "SIZE_OF_PARTITIONS": 100,
        "NUM_DATASET_PASSES": 50,
        "LEARNING_RATES_PHASE_1": [0.05, 0.01, 0.005, 0.1],
        "CHUNK_SIZE": 50,
        "training_csv": training_csv
    }
    url = 'http://localhost:5000/phase1'
    r = requests.post(url, json=body)
    phase1_res = r.json()
#     print(phase1_res)
    return phase1_res
    

In [5]:
def get_phase2_results(frbc,
                       validation_csv=None,
                       training_csv=None,
                       n_c=2,
                       chunk_size=100
                      ):
    body = {
        "algorithm": "classification",
        "Num_Classes": n_c,
        'FeatureRankingByClass': frbc,
        "CHUNK_SIZE": chunk_size,
        "NoOfModels_Ensemble": 3,
        "Max_FeatureSpaces_Size": 50,
        "train_crossvalidation_percentage": 1,
        "LEARNING_RATES_PHASE_2": [0.05, 0.001, 0.08, 0.1],
        "NET_LATTICE_PHASE_2": [3, 4, 5, 6, 7, 8],
        "no_data_Passes_kernel_Phase_2": 10,
        "NUM_DATASET_PASSES": 100,
        "validation_csv": validation_csv,
        "training_csv": training_csv
    }
    url = 'http://localhost:5000/phase2'
    r = requests.post(url, json=body)
    phase2_res = r.json()
    return phase2_res


In [6]:
def get_inference_results(testing_file):
    body = {
        "algorithm": "classification",
        "testing_file": testing_file
    }
    url = 'http://localhost:5000/inference'
    r = requests.post(url, json=body)
    inference_res = r.json()
    return inference_res

In [7]:
# def process_results(data_directory, dataset, seed, short, sep=','):
#     proc, gateway_proc, details = None, None, {}
#     try:
#         gateway_proc = subprocess.Popen(['python', os.path.join(data_directory, 'gateway_simulation.py')])
#         backend_proc = subprocess.Popen(['python', r"C:\Users\arpit\workspace\MainAppCuda\main.py"])
#         proc = subprocess.Popen(['python', os.path.join(data_directory, 'sonar_stream_V2.py'), dataset, str(seed), short, sep])
        
#         data_dir = r"C:\Users\arpit\workspace\setup-stuff\gateway_and_dataset\Classification\DATASETS_CLASSIFICATION\BINARY_PROBLEMS"
#         csv_path_train = os.path.join(data_dir, dataset, short + 'Train' + str(seed) + 'N_mod.txt')
#         csv_path_test = os.path.join(data_dir, dataset, short + 'Test' + str(seed) + 'N_mod.txt')
#         csv_path_val = os.path.join(data_dir, dataset, short + 'Val' + str(seed) + 'N_mod.txt')
        
#         train_df = pd.read_csv(csv_path_train, sep=sep)
#         target = list(train_df.iloc[:, -1].unique())
#         target = [str(tar) for tar in target]
        
#         prereq_res = get_prereq_results(target)
#         phase0_res = get_phase0_results()
#         phase1_res = get_phase1_results(len(target), csv_path_train)
#         phase2_res = get_phase2_results(phase1_res["class"], csv_path_val, csv_path_train)
#         inference_res = get_inference_results(csv_path_test)
        
#         features = min(len(phase1_res["class"][0]["rankings"]), 15)
#         details = {'dataset': dataset, 'seed': seed, 'KernelTime': phase2_res['kernelTime'], 'hostTime': phase2_res['hostTime']}
#         details['phase1_chunks_for_convergence'] = phase1_res['chunks_for_convergence']
#         details['phase2_chunks_for_convergence'] = phase2_res['chunks_for_convergence_phase2']
#         details['best_feature_list'] = phase2_res['minFeatureSpace']
#         details['best_lr'] = phase2_res['learning_rate']
#         details['best_fs'] = len(phase2_res['minFeatureSpace'])
#         details['model_id'] = phase2_res['model_index']
#         details['test_accuracy'] = inference_res['accuracyScore']
#         details['test_precision'] = inference_res['precision']
#         details['test_recall'] = inference_res['recall']
#         details['test_fscore'] = inference_res['fscore']
        
#         details['train_accuracy'] = phase2_res['accuracyScore_t']
#         details['train_precision'] = phase2_res['precision_t']
#         details['train_recall'] = phase2_res['recall_t']
#         details['train_fscore'] = phase2_res['fscore_t']
#         details['val_accuracy'] = phase2_res['accuracyScore']
#         details['val_precision'] = phase2_res['precision']
#         details['val_recall'] = phase2_res['recall']
#         details['val_fscore'] = phase2_res['fscore']
        
#         imp_features = []
#         for k in range(2):
#             for i in range(features):
#                 imp_features.append(phase1_res["class"][k]["rankings"][i]['Feature'])
#             details['important_features_class_'+ str(k)] = imp_features
#             imp_features = []
#         backend_proc.kill()
#         proc.kill()
#         gateway_proc.kill()
#     except:
#         traceback.print_exc()
#         if proc:
#             proc.kill()
#         if gateway_proc:
#             gateway_proc.kill()
#         if backend_proc:
#             backend_proc.kill()
#     return details


# dataset_dir =\
#     r"C:\Users\arpit\workspace\setup-stuff\gateway_and_dataset"
# datasets = ["Adult", "Breast Cancer", "Credit Screening", "Ionosphere", "Liver Disorder", "Pima Indian", "Sonar"]
# shorts = ["AD", "BC", "CR", "IO", "LD", "PI", "SN"] 
# header = [False, False, False, False, False, False, False]
# datasets = ["Ionosphere"]
# shorts = ["IO"]

# seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# for i in range(len(datasets)):
    
#     df = pd.DataFrame()
#     for seed in tqdm(seeds):
#         dataset = datasets[i]
#         short = shorts[i]
#         details = process_results(dataset_dir, dataset, seed, short)
#         df = df.append(details, ignore_index=True)
#         filepath = Path(dataset + 'highConf_.csv')
#         filepath.parent.mkdir(parents=True, exist_ok=True)
#         df.to_csv(filepath, index=False)


In [ ]:
def process_bin_clas(df_train, df_test):
    df_total = pd.concat([df_train, df_test], axis=0)
#     print("Merged shape:", df_total.shape)
    tar = df_total.iloc[:, -1]
    df_total = df_total.iloc[:, :-1]
    cat = df_total.select_dtypes(include=['object']).columns.to_list()
    df_total = pd.get_dummies(df_total, cat)
#     print("After dummies shape:", df_total.shape)
    df_total = pd.concat([df_total, tar], axis=1)
#     print("After concat shape:", df_total.shape)
    df_train = df_total.iloc[:df_train.shape[0],:]
    df_test = df_total.iloc[df_train.shape[0]:,:]
    
    return df_train, df_test

def get_missing():
    missingValueColumns = datafrm.columns[datafrm.isnull().any()].tolist()
    percent_missing = datafrm[missingValueColumns].isnull().sum()
    print("Missing value count columnwise:")
    print(percent_missing)


data_dir ="/home/asim/Desktop/Takshshila/IOT/TestScripts/DATASETS_CLASSIFICATION"




datasets = ['data11_cns_splits','data15_occupanydetection_splits','data19_musk_splits','data22_connect_splits',
            'data26_sales_splits','data2_letter_splits','data6_shuttle_splits', # 'data12_srbct_splits',
            'data16_poker_splits','data1_credit_splits','data23_forestcover_splits','data27_yeast_splits',
            'data7_penbased_splits','data13_lymphoma_splits','data17_skinseg_splits', 'data20_carvana_splits',
            'data24_voice_splits','data28_theorem_splits','data4_activity_splits', 'data8_income_splits',
            'data10_brain_splits','data14_prostrate_splits','data18_challenges_splits', 'data21_gimmecredit_splits',
            'data29_dota_splits','data5_bank_splits','data9_amlall_splits'] # 'data25_arrythmia_splits',
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for i in range(len(datasets)):
    print(datasets[i])
    for seed in tqdm(seeds):
        dataset = datasets[i]
        remove_split = "_".join(dataset.split("_")[:-1])
        #print(dataset, remove_split, seed)
        csv_path_train = os.path.join(data_dir, dataset, remove_split + '_train' + str(seed) + '.csv')
        csv_path_test = os.path.join(data_dir, dataset,  remove_split + '_test' + str(seed) + '.csv')
        df_train = pd.read_csv(csv_path_train)
        df_test = pd.read_csv(csv_path_test)
#         print("Before", "Train", df_train.shape, "Test", df_test.shape)
        df_train, df_test = process_bin_clas(df_train, df_test)
#         print("End", "Train", df_train.shape, "Test", df_test.shape)
        x, y = df_train.iloc[: , :-1], df_train.iloc[: , -1]
        if dataset == "data18_challenges_splits":
            x, y = df_train.iloc[: , 1:], df_train.iloc[: , 0]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=4, stratify=y)
        train_part_k = pd.concat([x_train, y_train], axis=1)
        train_part_rest = pd.concat([x_test, y_test], axis=1)
        
#         train_part_k.to_csv(os.path.join(data_dir, dataset, remove_split + '_train' + str(seed) + '_mod.csv'), index=False)
        df_train.to_csv(os.path.join(data_dir, dataset, remove_split + '_train' + str(seed) + '_mod.csv'), index=False)
        train_part_rest.to_csv(os.path.join(data_dir, dataset, remove_split + '_val' + str(seed) + '_mod.csv'), index=False)
        df_test.to_csv(os.path.join(data_dir, dataset, remove_split + '_test' + str(seed) + '_mod.csv'), index=False)
    

data11_cns_splits


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.01it/s]


data15_occupanydetection_splits


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.70it/s]

In [7]:
def process_results(data_directory, dataset, seed, sep=','):
    proc, gateway_proc, backend_proc, details = None, None, None, {}
    try:
        gateway_proc = subprocess.Popen(['python3', os.path.join(data_directory, 'gateway_simulation.py')])
        #backend_proc = subprocess.Popen(['python3', r"/home/asim/Desktop/Takshshila/IOT/MainAppCuda/main.py"])
        proc = subprocess.Popen(['python3', os.path.join(data_directory, 'sonar_stream_V2.py'), dataset, str(seed), "", sep])
        
        data_dir = '/home/asim/Desktop/Takshshila/IOT/TestScripts/DATASETS_CLASSIFICATION'
        remove_split = "_".join(dataset.split("_")[:-1])
        csv_path_train = os.path.join(data_dir, dataset, remove_split + '_train' + str(seed) + '_mod.csv')
        csv_path_test = os.path.join(data_dir, dataset, remove_split + '_test' + str(seed) + '_mod.csv')
        csv_path_val = os.path.join(data_dir, dataset, remove_split + '_val' + str(seed) + '_mod.csv')
        
        train_df = pd.read_csv(csv_path_train, sep=sep)
        target = list(train_df.iloc[:, -1].unique())
        target = [str(tar) for tar in target]
        
        prereq_res = get_prereq_results(target)
        phase0_res = get_phase0_results()
        phase1_res = get_phase1_results(len(target), csv_path_train)
        phase2_res = get_phase2_results(phase1_res["class"], csv_path_val, csv_path_train, len(target))
        inference_res = get_inference_results(csv_path_test)
        
        features = min(len(phase1_res["class"][0]["rankings"]), 15)
        details = {'dataset': dataset, 'seed': seed, 'KernelTime': phase2_res['kernelTime'], 'hostTime': phase2_res['hostTime']}
        details['phase1_chunks_for_convergence'] = phase1_res['chunks_for_convergence']
        details['phase2_chunks_for_convergence'] = phase2_res['chunks_for_convergence_phase2']
        details['best_feature_list'] = phase2_res['minFeatureSpace']
        details['best_lr'] = phase2_res['learning_rate']
        details['model_id'] = phase2_res['model_index']
        details['best_fs'] = len(phase2_res['minFeatureSpace'])
        details['test_accuracy'] = inference_res['accuracyScore']
        details['test_precision'] = inference_res['precision']
        details['test_recall'] = inference_res['recall']
        details['test_fscore'] = inference_res['fscore']
        details['test_accuracies'] = inference_res['test_accuracies']
        
        details['train_accuracy'] = phase2_res['accuracyScore_t']
        details['train_precision'] = phase2_res['precision_t']
        details['train_recall'] = phase2_res['recall_t']
        details['train_fscore'] = phase2_res['fscore_t']
        details['val_accuracy'] = phase2_res['accuracyScore']
        details['val_precision'] = phase2_res['precision']
        details['val_recall'] = phase2_res['recall']
        details['val_fscore'] = phase2_res['fscore']
        
        imp_features = []
        for k in range(2):
            for i in range(features):
                imp_features.append(phase1_res["class"][k]["rankings"][i]['Feature'])
            details['important_features_class_'+ str(k)] = imp_features
            imp_features = []
            
        '''
        if backend_proc:
            backend_proc.kill()
            
            
        '''
        if proc:
            proc.kill()
        if gateway_proc:
            gateway_proc.kill()
    except:
        traceback.print_exc()
        if proc:
            proc.kill()
        if gateway_proc:
            gateway_proc.kill()
        '''
        if backend_proc:
            backend_proc.kill()
        '''
    return details


data_dir ='/home/asim/Desktop/Takshshila/IOT/TestScripts'


# datasets = ['data15_occupanydetection_splits','data19_musk_splits','data26_sales_splits','data2_letter_splits',
#             'data12_srbct_splits', 'data27_yeast_splits', 'data7_penbased_splits', 'data13_lymphoma_splits',
#             'data17_skinseg_splits', 'data6_shuttle_splits', 'data16_poker_splits', 'data1_credit_splits',
#             'data20_carvana_splits', #-> Taking too long, Skipping it. 179k samples in training
#             'data23_forestcover_splits', #-> Taking too long, Skipping it. 154k samples in training
datasets = ['data24_voice_splits','data28_theorem_splits','data4_activity_splits', 'data8_income_splits',
            'data18_challenges_splits', 'data21_gimmecredit_splits', 'data22_connect_splits',
            'data5_bank_splits', 'data29_dota_splits']
# 'data25_arrythmia_splits' ,'data9_amlall_splits', 'data10_brain_splits','data14_prostrate_splits', 'data11_cns_splits',

seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for i in range(len(datasets)):
    df = pd.DataFrame()
    for seed in tqdm(seeds):
        dataset = datasets[i]
        details = process_results(data_dir, dataset, seed)
        df = df.append(details, ignore_index=True)
        filepath = Path(dataset + '_manasa.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

data24_voice_splits 1
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected


Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/home/asim/anaconda3/lib/python3.9/http/client.py", line 1371, in getresponse
    response.begin()
  File "/home/asim/anaconda3/lib/python3.9/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/home/asim/anaconda3/lib/python3.9/http/client.py", line 288, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handl

data24_voice_splits 2
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected
sonar disconnected


Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/home/asim/anaconda3/lib/python3.9/http/client.py", line 1371, in getresponse
    response.begin()
  File "/home/asim/anaconda3/lib/python3.9/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/home/asim/anaconda3/lib/python3.9/http/client.py", line 288, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handl

...

 10%|████████▎                                                                          | 1/10 [00:00<00:01,  6.90it/s]Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.requ

 30%|████████████████████████▉                                                          | 3/10 [00:00<00:01,  6.82it/s]Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.requ

 50%|█████████████████████████████████████████▌                                         | 5/10 [00:00<00:00,  6.50it/s]Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.requ

 70%|██████████████████████████████████████████████████████████                         | 7/10 [00:01<00:00,  6.31it/s]Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.requ

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:01<00:00,  6.48it/s]Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/asim/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 394, in _make_request
    conn.requ